# Import Libraries

In [1]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Linear Regression Model").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import IndexToString, StringIndexer

# Load Data

In [8]:
from pyspark import SparkFiles
# %% [markdown]
# # Load Data
# %%
url = 'https://raw.githubusercontent.com/Vivekfreak/Datasets/refs/heads/main/cruise_ship_info.csv'
spark.sparkContext.addFile(url)
data = spark.read.csv(SparkFiles.get("cruise_ship_info.csv"), header=True, inferSchema=True)

In [10]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
data.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

In [12]:
for item in data.head(1)[0]:
  print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [14]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [15]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [16]:
data.groupBy('Ship_name').count().show()

+------------+-----+
|   Ship_name|count|
+------------+-----+
|       Virgo|    1|
|     Fortuna|    1|
|      Shadow|    1|
|     Empress|    1|
|        Wind|    2|
|    Paradise|    1|
|        Surf|    1|
|      Wonder|    1|
|       Magic|    1|
|    Symphony|    1|
|    Sinfonia|    1|
| Inspiration|    1|
|   Millenium|    1|
|    Solstice|    1|
|PrideofAloha|    1|
|     Majesty|    2|
|     Ventura|    1|
|   Romantica|    1|
|      Spirit|    4|
|       Oasis|    1|
+------------+-----+
only showing top 20 rows



# Data Preprocessing

In [20]:
indexer = StringIndexer(inputCols=["Ship_name","Cruise_line"], outputCols=["Ship_name_Index","Cruise_line_Index"])
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Ship_name_Index|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           64.0|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           98.0|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|           27.0|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|           31.0|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|          

In [21]:
indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Ship_name_Index: double (nullable = false)
 |-- Cruise_line_Index: double (nullable = false)



In [23]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Ship_name_Index',
 'Cruise_line_Index']

In [25]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length', 'cabins', 'passenger_density', 'Ship_name_Index', 'Cruise_line_Index'],
                            outputCol='features')

In [26]:
output = assembler.transform(indexed)

In [27]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Ship_name_Index: double (nullable = false)
 |-- Cruise_line_Index: double (nullable = false)
 |-- features: vector (nullable = true)



In [28]:
print(output.features)

Column<'features'>


In [29]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Ship_name_Index=64.0, Cruise_line_Index=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 64.0, 16.0]))]

In [30]:
final_data = output.select('features','crew')

In [31]:
final_data.show()


+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



# Train Test Split

In [32]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [33]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               101|
|   mean| 7.722871287128725|
| stddev|3.6824332544212273|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [35]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                57|
|   mean| 7.920526315789474|
| stddev|3.1893480194976656|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



#Building Model

In [36]:
regressor = LinearRegression(labelCol='crew')

In [37]:
model=regressor.fit(train_data)

# Evaluation Model

In [38]:
pred_data = model.evaluate(test_data)

In [39]:
pred_data.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.8835185482382073|
| -0.7501876410265789|
|-0.47069334634649707|
| -0.5677605964489043|
| -0.5978066509114637|
|-0.28876382480618545|
|  0.3677859355780235|
|  -0.817608314740033|
|   0.620511977887892|
|  -0.789594258378095|
| -0.8184921114700501|
|   0.865380060319092|
| -0.4556659767857596|
|  1.7193726967640313|
| 0.12520230159145207|
| -0.8373188704919645|
| -1.5274983141561869|
|-0.44481602767837103|
|  0.7244221496445604|
|  0.6082075981972022|
+--------------------+
only showing top 20 rows



In [40]:
pred_data.rootMeanSquaredError

0.7965357760018978

In [41]:
pred_data.r2

0.936511717506245

In [42]:
pred_data.meanAbsoluteError

0.636528271478193

In [43]:
pred_data.meanSquaredError

0.6344692424509455

In [44]:
from pyspark.sql import functions as f
data.select(f.corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [45]:
data.select(f.corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [46]:
unlabeled_data = test_data.select('features')
test_predictions = model.transform(unlabeled_data)

In [47]:
test_predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,160.0,36.34,...|15.483518548238207|
|[6.0,30.276999999...| 4.300187641026579|
|[6.0,112.0,38.0,9...|11.370693346346497|
|[6.0,158.0,43.7,1...|14.167760596448904|
|[7.0,158.0,43.7,1...|14.197806650911463|
|[9.0,59.058,17.0,...| 7.688763824806186|
|[9.0,81.0,21.44,9...| 9.632214064421976|
|[9.0,85.0,19.68,9...| 9.507608314740033|
|[9.0,88.5,21.24,9...| 9.679488022112109|
|[9.0,90.09,25.01,...| 9.479594258378095|
|[9.0,105.0,27.2,8...| 11.49849211147005|
|[9.0,113.0,26.74,...|11.514619939680909|
|[9.0,116.0,26.0,9...| 11.45566597678576|
|[10.0,46.0,7.0,6....|2.7506273032359685|
|[10.0,77.0,20.16,...| 8.874797698408548|
|[10.0,105.0,27.2,...|11.517318870491964|
|[10.0,138.0,31.14...|13.377498314156187|
|[11.0,86.0,21.24,...| 9.744816027678372|
|[11.0,90.0,22.4,9...| 10.27557785035544|
|[11.0,108.977,26....|11.391792401802798|
+--------------------+------------